In [1]:
from tkinter import *
from tkinter import messagebox
from PIL import ImageTk, Image

In [2]:
class GUI(Frame):
    def __init__(self,master):
        '''Initialises board game'''
        Frame.__init__(self,master)
        self.master=master
        self.btn_arr=[[0 for i in range(8)] for j in range(8)]
        self.has_moven=[[False for i in range(16)],[False for i in range(16)]]
        self.chance,self.ind,self.promotion_status=0,None,False
        
        self.bg_color=['#C0C0C0','#404040']
        self.mv_color=['#99D9EA','#59C2DD']
        self.kl_color=['#FF7F27','#F26100']
        self.sl_color=['#FFF200','#FFB90E']
        self.piece_name=[[i for i in 'rnbqkbnrpppppppp'],[i for i in 'rnbqkbnrpppppppp']]
        self.game_track=[]
        self.piece_arr=self.create_pos()
        self.load_main_page()
        self.set_pieces()
        self.moves=move_class()
        self.move,self.kill,self.save,self.hidden,self.castling=self.moves.initialise_all(self.piece_arr,self.piece_name,self.chance,
                                                                            self.has_moven,self.master)
        
    def create_pos(self):
        '''Creates a position array for both colours'''
        row_order=[[0,1],[7,6]]
        pos_arr=[[],[]]
        for color in range(2):
            for col in range(16):
                pos_arr[color].append((row_order[color][col//8],col%8))
        return pos_arr
    
    def set_pieces(self):
        '''Places pieces in initial board position'''
        for color in [self.chance,self.chance^1]:
            for ind,pos in enumerate(self.piece_arr[color]):
                x,y=pos
                img=ImageTk.PhotoImage(Image.open("{}{}.png".format(self.piece_name[color][ind],color)))
                self.btn_arr[x][y].configure(image=img,width=70,height=70)
                self.btn_arr[x][y].image=img
                                       
        
    def load_main_page(self):
        '''Creates 64 buttons'''
        self.master.title('The game of CHESS')
        self.master.configure(background='black')
        self.master.geometry("870x570")
        self.master.resizable(0, 0)
        
        for i in range(8):
            for j in range(8):
                img = ImageTk.PhotoImage(Image.open("transparent_bg.png"))
                self.btn_arr[i][j] = Button(self.master, image=img, background=self.bg_color[(i+j)%2],relief=SUNKEN,
                                            activebackground=self.bg_color[(i+j)%2],name="({},{})".format(i,j))
                self.btn_arr[i][j].image=img
                self.btn_arr[i][j].bind("<Button-1>", self.onClick)
                self.btn_arr[i][j].place(x=2+70*j,y=2+70*i)
    
    def colourise(self,x,y):
        '''Colours move and kill buttons'''
        self.btn_arr[x][y].configure(bg=self.sl_color[(x+y)%2])
        for i,j in self.move[self.chance][self.ind]:
            self.btn_arr[i][j].configure(bg=self.mv_color[(i+j)%2])
        for i,j in self.kill[self.chance][self.ind]:
            self.btn_arr[i][j].configure(bg=self.kl_color[(i+j)%2])
            
    def decolourise(self,ind):
        '''Decolourises move, kill, and selected buttons'''
        x,y=self.piece_arr[self.chance][ind]
        self.btn_arr[x][y].configure(bg=self.bg_color[(x+y)%2])
        for i,j in self.move[self.chance][ind]:
            self.btn_arr[i][j].configure(bg=self.bg_color[(i+j)%2])
        for i,j in self.kill[self.chance][ind]:
            self.btn_arr[i][j].configure(bg=self.bg_color[(i+j)%2])
            
    def piece_shift(self,ind,i,j,x,y):
        self.game_track.append((self.piece_name[self.chance][ind],i,j,x,y))
        img=ImageTk.PhotoImage(Image.open("transparent_bg.png"))
        self.btn_arr[i][j].configure(image=img,width=70,height=70)
        self.btn_arr[i][j].image=img
        self.piece_arr[self.chance][ind]=(x,y)
        img=ImageTk.PhotoImage(Image.open("{}{}.png".format(self.piece_name[self.chance][ind],self.chance)))
        self.btn_arr[x][y].configure(image=img,width=70,height=70)
        self.btn_arr[x][y].image=img
        
    def pawn_promotion(self,pawn_ind):
        self.pawn_ind=pawn_ind
        self.promotion_status=True
        self.promo_value=StringVar()
        self.option=[]
        for img,pos in zip('qnrb',[(595,170),(705,170),(595,280),(705,280)]):
            image=ImageTk.PhotoImage(Image.open('{}{}.png'.format(img,self.chance)))
            self.option.append(Radiobutton(self.master,image=image,variable=self.promo_value,value=img))
            self.option[-1].image=image
            self.option[-1].place(x=0+pos[0],y=0+pos[1])
        self.promo_value.set('q')
        self.ok_button=Button(self.master,text="OK",command=self.unfreeze)
        self.ok_button.place(x=700,y=390)
        
    def unfreeze(self):
        self.promotion_status=False
        self.piece_name[self.chance^1][self.pawn_ind]=self.promo_value.get()
        x,y=self.piece_arr[self.chance^1][self.pawn_ind]
        img=ImageTk.PhotoImage(Image.open("{}{}.png".format(self.piece_name[self.chance^1][self.pawn_ind],self.chance^1)))
        self.btn_arr[x][y].configure(image=img,width=70,height=70)
        self.btn_arr[x][y].image=img
        self.move,self.kill,self.save,self.hidden,self.castling=self.moves.initialise_all(self.piece_arr,self.piece_name,self.chance^1,
                                                                                self.has_moven,self.master)
        for i in self.option:
            i.destroy()
        self.ok_button.destroy()
                
    def onClick(self,event):
        '''Triggered when a button is pressed'''
        if self.promotion_status==True:
            return
        x,y=eval(event.widget.winfo_name())
        if self.ind==None and ((x,y) in self.piece_arr[self.chance]):
            # only checking on pieces depending on self.chance
            for ind,pos in enumerate(self.piece_arr[self.chance]):
                if pos==(x,y):
                    self.ind=ind
                    print(self.ind)
                    print('Selected object \n move:{},kill:{},safe={},hidden={}'
                          .format(self.move[self.chance][self.ind],self.kill[self.chance][self.ind],
                                  self.save[self.chance][self.ind],self.hidden[self.chance][self.ind]))
            self.colourise(x,y)

        # If something is selected earlier and now clicked in one of its moveable positions execute below
        elif self.ind!=None and ((x,y) in self.move[self.chance][self.ind].union(self.kill[self.chance][self.ind])):
            if ((x,y) in self.kill[self.chance][self.ind]):
                for i,j in enumerate(self.piece_arr[self.chance^1]):
                    if j==(x,y):
                        self.piece_arr[self.chance^1][i]=None
                        break
                        
            self.decolourise(self.ind)
            i,j=self.piece_arr[self.chance][self.ind]
            self.piece_shift(self.ind,i,j,x,y)
            self.has_moven[self.chance][self.ind]=True
            if self.game_track[-1][0]=='p' and self.game_track[-1][3] in [0,7]:
                self.pawn_promotion(self.ind)
            if (x,y) in self.castling:
                if y==2:
                    self.piece_shift(0,x,0,x,y+1)
                elif y==6:
                    self.piece_shift(7,x,7,x,y-1)
            self.move,self.kill,self.save,self.hidden,self.castling=self.moves.initialise_all(self.piece_arr,self.piece_name,self.chance,
                                                                                self.has_moven,self.master)
            self.chance^=1
            self.ind=None
        elif self.ind!=None:
            self.decolourise(self.ind)
            self.ind=None

In [10]:
class move_class:
    def __init__(self):
        '''Initialises move_class object'''
        self.move=[[set() for i in range(16)],[set() for i in range(16)]]
        self.kill=[[set() for i in range(16)],[set() for i in range(16)]]
        self.save=[[set() for i in range(16)],[set() for i in range(16)]]
        self.hidden=[[set() for i in range(16)],[set() for i in range(16)]]
        self.colour={0:"WHITE",1:"BLACK"}
        self.status,self.status_msg=None,None
        
    def destroy_status(self):
        if self.status_msg!=None: self.status_msg.destroy()
        self.status_msg=None
    
    def initialise_all(self,obj_arr,obj_name,chance,has_moven,master):
        '''Updates the position of all the pieces in the board'''
        self.obj_arr=obj_arr
        self.chance=chance
        self.has_moven=has_moven
        self.master=master
        self.destroy_status()
        self.obj_name=obj_name
        
        for color in [chance,chance^1]:
            checkmate_status=True
            for ind in [0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,4]:
                pos=self.obj_arr[color][ind]
                if color==chance:
                    check_arr,pieces=set(),set()
                if pos!=None:
                    if self.obj_name[color][ind]=='k': t1,t2,t3,t4,check_status,castling=self.king(pos,color,self.obj_arr,
                                        direction_vect=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)],extend=2)
                    elif self.obj_name[color][ind]=='r': t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(1,0),(0,1),(-1,0),(0,-1)],extend=8)
                    elif self.obj_name[color][ind]=='n': t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(-1,2),(1,-2),(-1,-2)],
                                                             extend=2)
                    elif self.obj_name[color][ind]=='b': t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(1,1),(-1,1),(-1,-1),(1,-1)],extend=8)
                    elif self.obj_name[color][ind]=='q': t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(1,0),(0,1),(-1,0),(0,-1),(1,1),(-1,1),(-1,-1),(1,-1)],
                                                       extend=8)
                    elif self.obj_name[color][ind]=='p': t1,t2,t3,t4=self.pawn(pos,color,self.obj_arr,ind)
                    if len(t1)!=0 or len(t2)!=0:
                        checkmate_status=False
                    self.move[color][ind]=t1
                    self.kill[color][ind]=t2
                    self.save[color][ind]=t3
                    self.hidden[color][ind]=t4
            self.status=Label(self.master, text="STATUS", fg="#FFFFFF", bg="#000000", font=("Times", "36", "bold italic"))
            self.status.place(x=625,y=100)
            self.status_msg=Label(self.master, text="", fg="#FFFFFF", bg="#000000", font=("Times", "36", "bold italic"))
            self.status_msg.place(x=625,y=175)
            if check_status and not checkmate_status:
                self.status_msg.configure(text="{}\n in \n CHECK".format(self.colour[chance^1]))
            elif check_status and checkmate_status:
                self.status_msg.configure(text="{} \n WINS".format(self.colour[chance]))
            elif not check_status and checkmate_status:
                self.status_msg.configure(text="{} \n has \n no \n moves".format(self.colour[chance]))
            else:
                self.status_msg.destroy()
                self.status_msg=None
        return self.move,self.kill,self.save,self.hidden,castling

    def QBRN(self,pos,color,arr,direction_vect,extend):
        x,y=pos
        move,save,kill,hidden=set(),set(),set(),set()
        for i,j in direction_vect:
            save_kill_flag=False
            for k in range(1,extend):
                npos_x,npos_y=x+k*i,y+k*j
                if npos_x<0 or npos_y<0 or npos_y>7 or npos_x>7:
                    break
                if save_kill_flag==False and not ((npos_x,npos_y) in arr[0] or 
                                                  (npos_x,npos_y) in arr[1]):
                    move.add((npos_x,npos_y))
                elif save_kill_flag==False and (npos_x,npos_y) in arr[color]:
                    save_kill_flag=True
                    save.add((npos_x,npos_y))
                elif save_kill_flag==False and (npos_x,npos_y) in arr[color^1]:
                    save_kill_flag=True
                    kill.add((npos_x,npos_y))
                elif save_kill_flag==True:
                    hidden.add((npos_x,npos_y))
        return move,kill,save,hidden
    
    def pawn(self,pos,color,arr,ind):
        x,y=pos
        move,kill,save,hidden=set(),set(),set(),set()
        sign_bit=1-2*(color)
        if (x+sign_bit,y) not in arr[0]+arr[1]:
            move={(x+sign_bit,y)}
            if self.has_moven[color][ind]==False and (x+2*sign_bit,y) not in arr[0]+arr[1]:
                move.add((x+2*sign_bit,y))
        if y>=1:
            if (x+sign_bit,y-1) not in arr[0]+arr[1]:
                hidden.add((x+sign_bit,y-1))
            elif (x+sign_bit,y-1) in arr[color]:
                save.add((x+sign_bit,y-1))
            elif (x+sign_bit,y-1) in arr[color^1]:
                kill.add((x+sign_bit,y-1))
        if y<=6:
            if (x+sign_bit,y+1) not in arr[0]+arr[1]:
                hidden.add((x+sign_bit,y+1))
            elif (x+sign_bit,y+1) in arr[color]:
                save.add((x+sign_bit,y+1))
            elif (x+sign_bit,y+1) in arr[color^1]:
                kill.add((x+sign_bit,y+1))
        return move,kill,save,hidden
    
    def RBQ_check_arr_update(self,a,b):
        res=[]
        diff=max(abs(b[0]-a[0]),abs(b[1]-a[1]))
        step=((b[0]-a[0])//diff,(b[1]-a[1])//diff)
        for i in range(diff):
            res.append((b[0]-i*step[0],b[1]-i*step[1]))
        return res
    
    def king(self,pos,color,arr,direction_vect,extend):
        move,kill,save,hidden=self.QBRN(pos,color,arr,direction_vect,extend)
        check_status,castling=False,[]
        for ind,moves in enumerate(self.move[color^1][:8]):
            if ind==4:
                hidden.update(move.intersection(self.hidden[color^1][4]))
                hidden.update(move.intersection(self.move[color^1][4]))
                move.difference_update(self.hidden[color^1][4])
            else:
                intersect=move.intersection(moves)
                move.difference_update(intersect)
                hidden.update(intersect)
            
        for ind,hiddens in enumerate(self.hidden[color^1][8:]):
            intersect=move.intersection(hiddens)
            move.difference_update(intersect)
            hidden.update(intersect)
            
        for i in self.save[color^1]:
            intersect=kill.intersection(i)
            kill.difference_update(intersect)
            hidden.update(intersect)
        if color!=self.chance:
            check_status=self.check_for_check(pos,color)
            x,y=pos
            castling.append(self.check_for_castling(pos,color,4,0,[(x,1),(x,2),(x,3)],{(x,2),(x,3),(x,4)},(x,2)))
            castling.append(self.check_for_castling(pos,color,4,7,[(x,5),(x,6)],{(x,4),(x,5),(x,6)},(x,6)))
            if castling[1]==None: castling.pop(1)
            if castling[0]==None: castling.pop(0)
            move.update(castling)
        return move,kill,save,hidden,check_status,castling
    
    def check_for_castling(self,pos,color,king_pos,rook_pos,non_block_pos,non_check_pos,return_val):
        if self.has_moven[color][king_pos]==False and self.has_moven[color][rook_pos]==False:
            for i in non_block_pos:
                if i in self.obj_arr[color] or i in self.obj_arr[color^1]:
                    return
            for ind,oppo_pos in enumerate(self.obj_arr[color^1]):
                if oppo_pos==None:
                    continue
                if pos in self.kill[color^1][ind] or len(non_check_pos.intersection(self.move[color^1][ind]))!=0:
                    return
        return return_val
    
    def check_for_check(self,pos,color):
        check_arr,check_status=set(),False
        for ind,oppo_pos in enumerate(self.obj_arr[color^1]):
            if oppo_pos==None:
                continue
            if pos in self.kill[color^1][ind]:
                check_status=True
                if self.obj_name[color^1][ind] in ['n','p']:
                    check_arr.update(oppo_pos)
                if self.obj_name[color^1][ind] in ['r','b','q']:
                    check_arr.update(self.RBQ_check_arr_update(pos,oppo_pos))
            if pos in self.hidden[color^1][ind] and self.obj_name[color^1][ind] in ['r','b','q']:
                temp,count=self.RBQ_check_arr_update(pos,oppo_pos),0
                for check_pos in temp[1:]:
                    if check_pos in self.obj_arr[color^1]:
                        break
                    if check_pos in self.obj_arr[color]:
                        index=self.obj_arr[color].index(check_pos)
                        count+=1
                    if count==2:
                        break
                if count==1:
                    self.move[color][index].intersection_update(temp)
                    self.kill[color][index].intersection_update(temp)
        if len(check_arr)!=0:
            for ind,oppo_pos in enumerate(self.obj_arr[color]):
                self.move[color][ind].intersection_update(check_arr)
                self.kill[color][ind].intersection_update(check_arr)
        return check_status

In [11]:
root=Tk()
app=GUI(root)
app.mainloop()
del app

11
Selected object 
 move:{(2, 3), (3, 3)},kill:set(),safe=set(),hidden={(2, 4), (2, 2)}
12
Selected object 
 move:{(5, 4), (4, 4)},kill:set(),safe=set(),hidden={(5, 5), (5, 3)}
3
Selected object 
 move:{(1, 3), (2, 3)},kill:set(),safe={(1, 2), (3, 3), (1, 4), (0, 4), (0, 2)},hidden={(0, 1), (7, 3), (4, 7), (0, 0), (3, 0), (0, 7), (2, 1), (6, 3), (0, 6), (0, 5), (4, 3), (3, 6), (2, 5), (5, 3)}
5
Selected object 
 move:{(6, 4), (3, 1), (2, 0), (4, 2), (5, 3)},kill:set(),safe={(6, 6)},hidden={(5, 7)}
2
Selected object 
 move:{(1, 3), (4, 6), (5, 7), (2, 4), (3, 5)},kill:set(),safe={(1, 1)},hidden={(2, 0)}
6
Selected object 
 move:{(6, 4), (5, 5)},kill:{(5, 7)},safe=set(),hidden=set()
1
Selected object 
 move:{(2, 0), (1, 3), (2, 2)},kill:set(),safe=set(),hidden=set()
4
Selected object 
 move:{(6, 4), (7, 6), (7, 5)},kill:set(),safe={(6, 3), (7, 3), (6, 5)},hidden=set()
4
Selected object 
 move:{(0, 3), (1, 3), (0, 2)},kill:set(),safe={(1, 5), (0, 5), (1, 4)},hidden=set()


In [5]:
a,b=set([1,2,3]),set([4,5,6])
print(a.intersection(b))

set()


In [6]:
a,b={(1,2)},{(2,3)}
a.update(b)
print(a)

{(1, 2), (2, 3)}


In [7]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list() -> new empty list
 |  list(iterable) -> new list initialized from iterable's items
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __l